<h1 align=center> Proceso de ETL <sub align=center> (ETL PROCESS)</sub></h1>

<h3 align=left> Instalacion de librerias<sub>/Library installation</sub> </h3>

In [ ]:
%pip install -r requirements.txt

<h3 align=left> Importacion de librerias<sub>/Library Importing</sub> </h3>

In [ ]:
import pandas as pd
import ast
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

<h2 align=Center> Extraccion de datos<sub>/Data extraction</sub></h2>

<h3 align=left> Rutas /<sub> Paths</sub> </h3>

In [ ]:
# Ubicaciones de los archivos Json\\ Json files paths
raw_reviews = 'australian_user_reviews.json'
raw_items = 'australian_users_items.json'

<h3 align=left> Funciones<sub>/Functions</sub></h3>

In [ ]:
def Open_extraction(raw_data):
  """
  Esta función lee linea a linea  la data del archivo JSON y entrega una lista de diccionarios.
  This function reads each line of the JSON data from a file and converts it into a list of dictionaries.
  """
  new_list = []
  with open(raw_data, 'r', encoding='utf-8') as json_file:
      for line in json_file:
          new_dicc = ast.literal_eval(line) # ast.literal_eval convierte la línea en un diccionario / ast.literal_eval converts the line to a dictionary
          new_list.append(new_dicc)
  return new_list

In [ ]:
def analyzer(review):
  """
  Esta función analiza el sentimiento de una reseña de texto (review) y devuelve un valor que representa la polaridad del sentimiento
  This function analyze and assigns a score based on the sentiment analysis of the review
  """
  if isinstance(review, str):
      score = SentimentIntensityAnalyzer().polarity_scores(review)
      if score['compound'] < -0.05:
          return 0
      elif score['compound'] > 0.05:
          return 2
      else:
          return 1

In [ ]:
def Unnesting_data(raw_df,data_column):
    """
    Esta función transforma una columna con datos anidados en un conjunto de columnas independientes y las une en un solo Dataframe
    This function transforms a column with nested data into a set of independent columns and joins them into a single Dataframe.
    """
    data_columns = raw_df.explode([data_column])
    unnested_data1 =pd.json_normalize(data_columns[data_column]).set_index(data_columns[data_column].index)
    processed_data = pd.concat([data_columns, unnested_data1], axis=1).reset_index().drop(['index',data_column], axis = 1)
    return processed_data


In [ ]:
Items_nested_data = pd.DataFrame(Open_extraction(raw_items))
Items_nested_data

In [ ]:
reviews_nested_data = pd.DataFrame(Open_extraction(raw_reviews))
reviews_nested_data

<h2 align=Center> Transformacion de datos<sub>/Data transform</sub></h2>

<h3 align=left> Desanidado de columnas<sub>/unnesting columns</sub></h3>

Items

In [ ]:
Items_unnested_data = Unnesting_data(Items_nested_data,'items')
preprocess_items = Items_unnested_data

reviews

In [ ]:
reviews_unnested_data = Unnesting_data(reviews_nested_data,'reviews')
preprocess_reviews = reviews_unnested_data

<h3>Datos nulos<sub>/null data<sub></h3>

In [ ]:
#Columnas de data items con datos nulos / Identifying columns with null values in items data.
Items_unnested_data.isnull().any()

In [ ]:
#Columnas de data reviews con datos nulos /Identifying columns with null values in reviews data.
reviews_unnested_data.isnull().any()

<h4>Datos nulos por dataframe<sub>/Null data by df</h4>

In [ ]:
#Proporcion de valores por variable data de items
(
    preprocess_items
    .isnull()
    .melt(value_name='missing')
    .pipe(
        lambda df: (
            sns.displot(
                data=df,
                y='variable',
                hue='missing',
                multiple='fill',
                aspect=2
            )
        )
    )
)

In [ ]:
#Proporcion de valores por variable data de reviews
(
    preprocess_reviews
    .isnull()
    .melt(value_name='missing')
    .pipe(
        lambda df: (
            sns.displot(
                data=df,
                y='variable',
                hue='missing',
                multiple='fill',
                aspect=2
            )
        )
    )
)

 <h4>Eliminacion de nulos<sub>/Removing null values</sub></h4>

In [ ]:
#Eliminacion de datos nulos datos de items y reviews / Eliminating null data in items and reviews.
preprocess_reviews = preprocess_reviews.dropna()
preprocess_items = preprocess_items.dropna()

<h4>Preparacion de columnas<sub>/Columns preparation</sub></h4>

In [ ]:
#Creacion de columna 'year' / Creating a 'year' column
preprocess_reviews ['year'] = preprocess_reviews['posted'].str.extract('(\d{4})', expand=False)
preprocess_reviews

In [ ]:
preprocess_reviews['recommend'].isna().sum()

In [ ]:
#Cambiamos los elementos de la columna recommend por unos mas faciles de procesar por el modelo /Replacing values in the "recommend" column with values easier for the model to process.
preprocess_reviews['recommend'] = preprocess_reviews['recommend'].replace({True: 1, False: 0}).astype(int)
preprocess_reviews

<h3 align=left> Analisis de sentimientos<sub>/Sentiment analyzer</sub></h3>

In [ ]:
nltk.download('punkt')
nltk.download('vader_lexicon')

In [ ]:
preprocess_reviews['sentiment_analysis'] = preprocess_reviews['review'].apply(analyzer)
preprocess_reviews

<h4>Eliminacion de columnas no necesarias para el analisis<sub>/Removing columns not needed for analysis.</sub></h4>

In [ ]:
processed_reviews = preprocess_reviews.drop(['user_url','last_edited','funny','helpful','posted','review'], axis = 1)
processed_reviews

In [ ]:
processed_items = preprocess_items.drop(['items_count','user_url','playtime_2weeks'], axis = 1)
processed_items

<h2 align=Center> Carga de datos<sub>/Data load</sub></h2>

<h4>Convercion de archivos Json a parquet<sub>/Converting JSON files to Parquet format<sub></h4>

In [ ]:
processed_items = processed_items.to_parquet('items.parquet')


In [ ]:
processed_reviews = processed_reviews.to_parquet('reviews.parquet')
